# Beginner of Natural Language
I saw the beatuiful kerenl written by Howard'. It was so readable but I can't correctly catch the menaing of each function since I'm a beginner of beginner of NLP. So that I make this kerenl to interpret the each line.  

** I really thanks for sharing the original code Howard in Kaggle**  
*If it is unpolite and unlegal way, I delete it in soon*

### Bold Setence is attached by me ###

In [1]:
import sys, os, re, csv, codecs, numpy as np, pandas as pd

from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.layers import Dense, Input, LSTM, Embedding, Dropout, Activation
from keras.layers import Bidirectional, GlobalMaxPool1D
from keras.models import Model
from keras import initializers, regularizers, constraints, optimizers, layers

path = '../input/'
comp = 'jigsaw-toxic-comment-classification-challenge/'
EMBEDDING_FILE=f'{path}glove6b50d/glove.6B.50d.txt'
TRAIN_DATA_FILE=f'{path}{comp}train.csv'
TEST_DATA_FILE=f'{path}{comp}test.csv'

Set some basic config parameters & Read in our data and replace missing values:

In [2]:
embed_size = 50 # how big is each word vector
max_features = 20000 # how many unique words to use (i.e num rows in embedding vector)
maxlen = 100 # max number of words in a comment to use
train = pd.read_csv(TRAIN_DATA_FILE)
test = pd.read_csv(TEST_DATA_FILE)

list_sentences_train = train["comment_text"].fillna("_na_").values
list_classes = ["toxic", "severe_toxic", "obscene", "threat", "insult", "identity_hate"]
y = train[list_classes].values
list_sentences_test = test["comment_text"].fillna("_na_").values

Standard keras preprocessing, to turn each comment into a list of word indexes of equal length (with truncation or padding as needed).

In [3]:
tokenizer = Tokenizer(num_words = max_features)
tokenizer.fit_on_texts(list(list_sentences_train))
list_tokenized_train = tokenizer.texts_to_sequences(list_sentences_train)
list_tokenized_test = tokenizer.texts_to_sequences(list_sentences_test)

**What is acted behind the function, tokenizer? I want to see it :)**

In [4]:
train_list = [['I', 'love', 'you', 'all'], ['I', 'love', 'love', 'favor', 'of', 'you']]
tokenizer_test = Tokenizer(num_words = 100)
tokenizer_test.fit_on_texts(train_list)
train_tokenized_list = tokenizer_test.texts_to_sequences(train_list)
print(train_tokenized_list)

**Did you see the 'love' which appeared three times (the frequentists) indexed as 1 and the I, you appeared two times indexed by 2,3, on the order in the sentence.**

In [5]:
print('Train')
print('row len :', len(list_tokenized_train))
print('max word num :', max([len(row) for row in list_tokenized_train]))
print('min word num : ', min([len(row) for row in list_tokenized_train]))
print('Test')
print('row len :', len(list_tokenized_test))
print('max word num :', max([len(row) for row in list_tokenized_test]))
print('min word num : ', min([len(row) for row in list_tokenized_test]))

In [6]:
X_t = pad_sequences(list_tokenized_train, maxlen=maxlen)
X_te = pad_sequences(list_tokenized_test, maxlen=maxlen)

** What happend by pad_sequnces and max_len?**

In [7]:
print('Number of Word: ', len(list_tokenized_train[0]))
print(list_tokenized_train[0])
print('Number of Element: ',X_t[0].shape[0])
print(X_t[0])

100 - 47 = 53 zero appended to left

In [8]:
ix_over_100 = -1
for i in range(len(list_tokenized_train)):
    if len(list_tokenized_train[i]) > 100:
        ix_over_100 = i
        break
print('Number of Word: ', len(list_tokenized_train[ix_over_100]))
print(list_tokenized_train[ix_over_100])
print('Number of Element: ',X_t[ix_over_100].shape[0])
print(X_t[ix_over_100])

Cut off the first 14 elements. So what is the first 14 elements? 

In [9]:
train_list = [['I', 'love', 'you', 'all'], ['I', 'am', 'in', 'favor', 'of', 'you']]
tokenizer_test = Tokenizer(num_words = 100)
tokenizer_test.fit_on_texts(train_list)
train_tokenized_list = tokenizer_test.texts_to_sequences(train_list)
print(train_tokenized_list)
t_X = pad_sequences(train_tokenized_list, maxlen=2)
print(t_X)

**When max_len = 2, the pad_sequences leave the end of two words. So that when 114 len and 100 max_len, cut off the first 14 words :)**

Read the glove word vectors (space delimited strings) into a dictionary from word->vector.

In [10]:
def get_coefs(word,*arr): return word, np.asarray(arr, dtype='float32')
embeddings_index = dict(get_coefs(*o.strip().split()) for o in open(EMBEDDING_FILE))

**What coefs inserted into the embeddings_index?**

In [11]:
for i, (key, val) in enumerate(embeddings_index.items()):
    print(key, len(val))
    print('the 5 coefficetions for {}: {}'.format(key, val[:5]))
    if i == 4: break

** Embedding File name is 'glove.6B.50d.txt.' It looks like that the glove.XB.Yd.txt, X is number of words, and Y is coefficient dimensions.**

Use these vectors to create our embedding matrix, with random initialization for words that aren't in GloVe. We'll use the same mean and stdev of embeddings the GloVe has when generating the random init.

In [12]:
all_embs = np.stack(embeddings_index.values())
emb_mean,emb_std = all_embs.mean(), all_embs.std()
emb_mean,emb_std

**What happend behind np.stack(embeddings_index.values())?**  
embeddings_index is a dictionary. I try to experiment by a small dictionary

In [13]:
test = {'a':[1,2,3], 'b':[4,5,6]}
print(test.values())
print(np.stack(test.values()))

**np.stack helped to make a matrix 60B word x 50 dimensions of coefficents.**

In [14]:
word_index = tokenizer.word_index
nb_words = min(max_features, len(word_index))
embedding_matrix = np.random.normal(emb_mean, emb_std, (nb_words, embed_size))
for word, i in word_index.items():
    if i >= max_features: continue
    embedding_vector = embeddings_index.get(word)
    if embedding_vector is not None: embedding_matrix[i] = embedding_vector

**Interpreation**
- tokenizer.word_index is a function getting an index, as alluded above, which measured by {'the most appearance':1, 'the 2nd appearance':2, ...}
- nb_words: min(the limitation of the system(tokenzier), the unique number of words) )
- embedding_matrix: get random coefficients by normal distribution, the size (nb_words X embed_size) where embed_size: neuron numbers(Is it correct? I'm beignner)
- for func: assign a correct vector mathced with embedding_index of Glove, but a random embedding values if word_index >= max_features == the out of bound, 

---

Simple bidirectional LSTM with two fully connected layers. We add some dropout to the LSTM since even 2 epochs is enough to overfit.

In [15]:
inp = Input(shape=(maxlen,))
x = Embedding(max_features, embed_size, weights=[embedding_matrix])(inp)
x = Bidirectional(LSTM(50, return_sequences=True, dropout=0.1, recurrent_dropout=0.1))(x)
x = GlobalMaxPool1D()(x)
x = Dense(50, activation="relu")(x)
x = Dropout(0.1)(x)
x = Dense(6, activation="sigmoid")(x)
model = Model(inputs=inp, outputs=x)
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

**Interpretation**  
- embedding: Initialize the embedding word matrix, weighting to each word
- Bidirectional: A train model by two direction forward and backward
- MaxPool: Extract the effective node
- Dense, activation = 'relu', sigmoid': Get the nonlinear values
- Dropout: Shrinkage the node size in random

---

Now we're ready to fit out model! Use `validation_split` when not submitting.

In [16]:
#model.fit(X_t, y, batch_size=32, epochs=2, validation_split=0.1);

And finally, get predictions for the test set and prepare a submission CSV:

In [17]:
#y_test = model.predict([X_te], batch_size=1024, verbose=1)
#sample_submission = pd.read_csv(f'{path}{comp}sample_submission.csv')
#sample_submission[list_classes] = y_test
#sample_submission.to_csv('submission.csv', index=False)